 ### Import Packages

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics

### Loading the Dataset

In [2]:
df = pd.read_csv("ipl_csv2/all_matches.csv")
pd.set_option("display.max_columns", None)
df.head()

c:\users\prash\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Converting the Complete Date into Year Format Only

In [3]:
from datetime import datetime
df['start_date'] = df['start_date'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y'))
df['start_date'] = df['start_date'].dt.year
df["start_date"] = df["start_date"].astype(int)
print(type(df["start_date"]))
df.head()

<class 'pandas.core.series.Series'>


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2007/08,2008,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2007/08,2008,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2007/08,2008,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Feature Selection
> Only feature to be used are 

> ["match_id","start_date" ,"ball" , "venue", "innings", "batting_team", "bowling_team", "striker", "non_striker","bowler", "runs_off_bat","extras"]

In [4]:
features = ["match_id","start_date" ,"ball" , "venue", "innings", "batting_team", "bowling_team",
            "striker", "non_striker","bowler", "runs_off_bat","extras"]
df = df[features]
df.head()

,match_id,start_date,ball,venue,innings,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras
0,335982,2008,0.1,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1
1,335982,2008,0.2,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0
2,335982,2008,0.3,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1
3,335982,2008,0.4,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0
4,335982,2008,0.5,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0


### Some Feature Engineering

In [5]:
#Combining the runs off the bat and extras
df["total_run"] = df["runs_off_bat"] + df["extras"]
df = df.drop(columns=['runs_off_bat','extras'])

In [6]:
df.head(4)

,match_id,start_date,ball,venue,innings,batting_team,bowling_team,striker,non_striker,bowler,total_run
0,335982,2008,0.1,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,1
1,335982,2008,0.2,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0
2,335982,2008,0.3,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,1
3,335982,2008,0.4,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0


 ### Drop Teams which dont play now and Change the names of the Fews Teams
> Teams to drop => ["Deccan Chargers", "Kochi Tuskers Kerala", "Pune Warriors","Rising Pune Supergiant","Gujarat Lions"]

> Teams to change the name:
> 1. Delhi Daredevils -> Delhi Capitals
> 2. Kings XI Punjab -> Punjab Kings

In [7]:
#Drop Teams
for team in ["Deccan Chargers", "Kochi Tuskers Kerala", "Pune Warriors","Rising Pune Supergiant","Rising Pune Supergiants","Gujarat Lions"]:
    df = df[df["batting_team"] != team]
    df = df[df["bowling_team"] != team] 

In [8]:
#Change the name of the teams
df.loc[(df.batting_team == 'Delhi Daredevils'),'batting_team']='Delhi Capitals'
df.loc[(df.bowling_team == 'Delhi Daredevils'),'bowling_team']='Delhi Capitals'

df.loc[(df.batting_team == 'Kings XI Punjab'),'batting_team']='Punjab Kings'
df.loc[(df.bowling_team == 'Kings XI Punjab'),'bowling_team']='Punjab Kings'


In [9]:
df["batting_team"].value_counts()

Mumbai Indians                 21278
Kolkata Knight Riders          19886
Delhi Capitals                 19766
Punjab Kings                   19701
Royal Challengers Bangalore    19621
Chennai Super Kings            19381
Rajasthan Royals               17136
Sunrisers Hyderabad            13692
Name: batting_team, dtype: int64

### Removing the Overs Greater than 6.0

In [10]:
df = df[df['ball'] <= 6.0]
df.shape

(47466, 11)

#### Repetative Names used for the stadiums
1. MA Chidambaram Stadium, Chepauk & "MA Chidambaram Stadium, Chepauk, Chennai" --> "MA Chidambaram Stadium"  
2. "M.Chinnaswamy Stadium" --> M Chinnaswamy Stadium  
3. "Wankhede Stadium, Mumbai" --> Wankhede Stadium  
4. Sardar Patel Stadium, Motera --> Narendra Modi Stadium
5. Feroz Shah Kotla --> "Arun Jaitley Stadium'

In [11]:
# Previous Venue Names
df.venue.value_counts()

Eden Gardens                                            5081
Wankhede Stadium                                        4661
M Chinnaswamy Stadium                                   3941
Feroz Shah Kotla                                        3572
Sawai Mansingh Stadium                                  3074
MA Chidambaram Stadium, Chepauk                         3003
Dubai International Cricket Stadium                     2516
Punjab Cricket Association Stadium, Mohali              2241
Sheikh Zayed Stadium                                    2185
Rajiv Gandhi International Stadium, Uppal               1885
Sharjah Cricket Stadium                                 1343
Rajiv Gandhi International Stadium                      1120
M.Chinnaswamy Stadium                                   1096
Arun Jaitley Stadium                                    1039
Kingsmead                                                899
Sardar Patel Stadium, Motera                             825
Brabourne Stadium       

In [12]:
#Changing Venue Names
df.loc[(df.venue == 'MA Chidambaram Stadium, Chepauk'),'venue']='MA Chidambaram Stadium'
df.loc[(df.venue == 'MA Chidambaram Stadium, Chepauk, Chennai'),'venue']='MA Chidambaram Stadium'

df.loc[(df.venue == 'M.Chinnaswamy Stadium'),'venue']='M Chinnaswamy Stadium'

df.loc[(df.venue == 'Wankhede Stadium, Mumbai'),'venue']='Wankhede Stadium'

df.loc[(df.venue == 'Sardar Patel Stadium, Motera'),'venue']='Narendra Modi Stadium'

df.loc[(df.venue == 'Feroz Shah Kotla'),'venue']='Arun Jaitley Stadium'

In [13]:
#did not drop any row just changes the duplicate names
df.shape

(47466, 11)

### Making Groups of the Data

In [16]:
grouped_data = df.groupby(["batting_team"])
grouped_data.first()

,match_id,start_date,ball,venue,innings,bowling_team,striker,non_striker,bowler,total_run
batting_team,,,,,,,,,,
Chennai Super Kings,335983,2008,0.1,"Punjab Cricket Association Stadium, Mohali",1,Punjab Kings,PA Patel,ML Hayden,B Lee,0
Delhi Capitals,335984,2008,0.1,Arun Jaitley Stadium,2,Rajasthan Royals,G Gambhir,V Sehwag,MM Patel,0
Kolkata Knight Riders,335982,2008,0.1,M Chinnaswamy Stadium,1,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,1
Mumbai Indians,335985,2008,0.1,Wankhede Stadium,1,Royal Challengers Bangalore,L Ronchi,ST Jayasuriya,P Kumar,0
Punjab Kings,335983,2008,0.1,"Punjab Cricket Association Stadium, Mohali",2,Chennai Super Kings,K Goel,JR Hopes,JDP Oram,4
Rajasthan Royals,335984,2008,0.1,Arun Jaitley Stadium,1,Delhi Capitals,T Kohli,YK Pathan,GD McGrath,0
Royal Challengers Bangalore,335982,2008,0.1,M Chinnaswamy Stadium,2,Kolkata Knight Riders,R Dravid,W Jaffer,AB Dinda,1
Sunrisers Hyderabad,598004,2013,0.1,"Rajiv Gandhi International Stadium, Uppal",2,Royal Challengers Bangalore,PA Reddy,PA Patel,JD Unadkat,1


> #### Aanalysing the Grouped Data 

In [17]:
csk = grouped_data.get_group("Chennai Super Kings")
csk.head()

,match_id,start_date,ball,venue,innings,batting_team,bowling_team,striker,non_striker,bowler,total_run
225,335983,2008,0.1,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Punjab Kings,PA Patel,ML Hayden,B Lee,0
226,335983,2008,0.2,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Punjab Kings,PA Patel,ML Hayden,B Lee,0
227,335983,2008,0.3,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Punjab Kings,PA Patel,ML Hayden,B Lee,1
228,335983,2008,0.4,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Punjab Kings,ML Hayden,PA Patel,B Lee,0
229,335983,2008,0.5,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Punjab Kings,ML Hayden,PA Patel,B Lee,4


>> This Displays all the matches of CSK as batting Team

In [18]:
#Code to get innings of any match id 
print("The innings of match with match id 33598 is: ",csk[csk["match_id"] == 335983]['innings'].unique()[0])

#Code for total powerplay socre by any team"
print("The total powerplay score of match with match id 33598 is: ",csk[csk["match_id"] == 335983]['total_run'].sum())

##Number of Batsmen that played in Powerplay
print("The batsmen played in match with match id 33598 is: ",csk[csk["match_id"] == 335983]['striker'].unique())

The innings of match with match id 33598 is:  1
The total powerplay score of match with match id 33598 is:  53
The batsmen played in match with match id 33598 is:  ['PA Patel' 'ML Hayden' 'MEK Hussey']


### Making the Custom DataSet in the Format we require to make the Predictions.
> Here, we need to predict the final score of powerplay given the inputs and thus we will make a dataset with final powerplay score in it as the target variables given all the inputs as feature variables

In [19]:
# Making the List of all the teams which played in ILP 2021
teams = ['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Punjab Kings', 'Rajasthan Royals',
       'Delhi Capitals', 'Mumbai Indians', 'Sunrisers Hyderabad']

In [22]:
final_data = pd.DataFrame(columns=["match_id","start_date","venue","innings","batting_team",
                                   "bowling_team","score", "no_of_batsmen"])

i = 0
for team in teams:
    t = grouped_data.get_group(team)
    match_id = np.array(t.match_id.unique())
    for ID in match_id:
        score = t[t["match_id"] == ID]['total_run'].sum()
        bowling_team = t[t["match_id"] == ID]['bowling_team'].unique()[0]
        batting_team = team
        innings = t[t["match_id"] == ID]['innings'].unique()[0]
        venue = t[t["match_id"] == ID]['venue'].unique()[0]
        start_date = t[t["match_id"] == ID]['start_date'].unique()[0]
        no_of_batsmen = len(t[t["match_id"] == ID]['striker'].unique())
        
        final_data.loc[i,"match_id"] = ID
        final_data.loc[i,"start_date"] = start_date
        final_data.loc[i,"venue"] = venue
        final_data.loc[i,"innings"] = innings
        final_data.loc[i,"batting_team"] = batting_team
        final_data.loc[i,"bowling_team"] = bowling_team
        final_data.loc[i,"no_of_batsmen"] = no_of_batsmen
        final_data.loc[i,"score"] = score
        i = i+1
        
final_data.head()

,match_id,start_date,venue,innings,batting_team,bowling_team,score,no_of_batsmen
0,335982,2008,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,61,3
1,335993,2008,MA Chidambaram Stadium,1,Kolkata Knight Riders,Chennai Super Kings,61,5
2,335997,2008,Eden Gardens,1,Kolkata Knight Riders,Mumbai Indians,40,5
3,336000,2008,Sawai Mansingh Stadium,2,Kolkata Knight Riders,Rajasthan Royals,51,4
4,336003,2008,"Punjab Cricket Association Stadium, Mohali",2,Kolkata Knight Riders,Punjab Kings,31,6


#### Analysing the final dataset 

In [25]:
# Managing Few Datatypes
final_data['score'] = final_data['score'].astype('int')
final_data['innings'] = final_data['innings'].astype('int')
final_data['no_of_batsmen'] = final_data['no_of_batsmen'].astype('int')
print("The datatype of no_of_batsmen is: ", final_data['no_of_batsmen'].dtype)
print("The datatype of score is: ", final_data['score'].dtype)
print("The datatype of innings is: ", final_data['innings'].dtype)

The datatype of no_of_batsmen is:  int32
The datatype of score is:  int32
The datatype of innings is:  int32


In [26]:
#Score of both the teams in one match
final_data[final_data["match_id"] == 335982] 

,match_id,start_date,venue,innings,batting_team,bowling_team,score,no_of_batsmen
0,335982,2008,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,61,3
169,335982,2008,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,26,6


In [27]:
# Average Power Play score by any team
final_data[final_data.venue == 'M Chinnaswamy Stadium']["score"].mean()

45.992592592592594

### Encoding the Categorical Values in dataset

#### Deciding the Venue Priority for Ordinal Encoding

In [28]:
avg_score_venue = {}
for ven in  final_data.venue.unique():
    avg_score = final_data[final_data.venue == ven]["score"].mean()
    avg_score_venue[ven] = avg_score
sort_orders = sorted(avg_score_venue.items(), key=lambda x: x[1])
print(sort_orders)
priority = 1
venue_priority = {}
for ven in sort_orders:
    venue_priority[ven[0]] = priority
    priority+=1
venue_priority

[('Buffalo Park', 33.0), ('OUTsurance Oval', 33.5), ('De Beers Diamond Oval', 35.5), ('Shaheed Veer Narayan Singh International Stadium', 38.0), ('Himachal Pradesh Cricket Association Stadium', 38.5), ('JSCA International Stadium Complex', 39.0), ('Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 42.1875), ('Dr DY Patil Sports Academy', 42.3), ('Kingsmead', 43.166666666666664), ('New Wanderers Stadium', 43.7), ('SuperSport Park', 43.75), ('Sheikh Zayed Stadium', 44.13793103448276), ('Newlands', 44.3), ('Sawai Mansingh Stadium', 44.75609756097561), ('Barabati Stadium', 45.0), ('Maharashtra Cricket Association Stadium', 45.45), ('Dubai International Cricket Stadium', 45.60606060606061), ('M Chinnaswamy Stadium', 45.992592592592594), ('Eden Gardens', 45.99264705882353), ('Rajiv Gandhi International Stadium, Uppal', 46.12), ("St George's Park", 46.5), ('MA Chidambaram Stadium', 46.78431372549019), ('Wankhede Stadium', 46.88095238095238), ('Sharjah Cricket Stadium', 47.02777777777778),

{'Buffalo Park': 1,
 'OUTsurance Oval': 2,
 'De Beers Diamond Oval': 3,
 'Shaheed Veer Narayan Singh International Stadium': 4,
 'Himachal Pradesh Cricket Association Stadium': 5,
 'JSCA International Stadium Complex': 6,
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 7,
 'Dr DY Patil Sports Academy': 8,
 'Kingsmead': 9,
 'New Wanderers Stadium': 10,
 'SuperSport Park': 11,
 'Sheikh Zayed Stadium': 12,
 'Newlands': 13,
 'Sawai Mansingh Stadium': 14,
 'Barabati Stadium': 15,
 'Maharashtra Cricket Association Stadium': 16,
 'Dubai International Cricket Stadium': 17,
 'M Chinnaswamy Stadium': 18,
 'Eden Gardens': 19,
 'Rajiv Gandhi International Stadium, Uppal': 20,
 "St George's Park": 21,
 'MA Chidambaram Stadium': 22,
 'Wankhede Stadium': 23,
 'Sharjah Cricket Stadium': 24,
 'Narendra Modi Stadium': 25,
 'Punjab Cricket Association Stadium, Mohali': 26,
 'Arun Jaitley Stadium': 27,
 'Subrata Roy Sahara Stadium': 28,
 'Punjab Cricket Association IS Bindra Stadium, Mohali': 29,


#### Deciding the batting team priority manually

In [29]:
#Batting Priority Manual
batting_priority_manual = {
    'Chennai Super Kings': 5,
     'Royal Challengers Bangalore': 8,
     'Delhi Capitals': 7,
     'Rajasthan Royals': 1,
     'Mumbai Indians': 2,
     'Punjab Kings': 3,
     'Sunrisers Hyderabad': 6,
     'Kolkata Knight Riders': 4
}

#### Deciding the batting team priority manually

In [30]:
#Bowling Priority Manual
bowling_priority_manual = {
     'Chennai Super Kings': 8,
     'Royal Challengers Bangalore': 4,
     'Delhi Capitals': 3,
     'Rajasthan Royals': 1,
     'Mumbai Indians': 6,
     'Punjab Kings': 5,
     'Sunrisers Hyderabad': 7,
     'Kolkata Knight Riders':2
    
}

#### Performing the Ordinal Encoding Using the Orders defined earlier

In [31]:
#Creatinf the Copy of our dataset, this will be used by the model and so it's names is defined as model_data
model_data = final_data.copy()

#Match Id and Start Date is not needed and hence we are dropping it here
model_data= model_data.drop(columns=['match_id','start_date'])

model_data.head()

,venue,innings,batting_team,bowling_team,score,no_of_batsmen
0,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,61,3
1,MA Chidambaram Stadium,1,Kolkata Knight Riders,Chennai Super Kings,61,5
2,Eden Gardens,1,Kolkata Knight Riders,Mumbai Indians,40,5
3,Sawai Mansingh Stadium,2,Kolkata Knight Riders,Rajasthan Royals,51,4
4,"Punjab Cricket Association Stadium, Mohali",2,Kolkata Knight Riders,Punjab Kings,31,6


**Note: Here we have dropped the start date, however the same can be kept and used for test train split ahead** 

In [32]:
#Ordinal Encoding 
ordinal_enc_model_data = model_data.copy()
ordinal_enc_model_data["venue_enc"] = ordinal_enc_model_data["venue"].map(venue_priority)
ordinal_enc_model_data["batting_team_enc"] = ordinal_enc_model_data["batting_team"].map(batting_priority_manual)
ordinal_enc_model_data["bowling_team_enc"] = ordinal_enc_model_data["bowling_team"].map(bowling_priority_manual)

#Dropping the Categorical Data
ordinal_enc_model_data = ordinal_enc_model_data.drop(columns=['venue','batting_team', 'bowling_team'])
ordinal_enc_model_data.head()

,innings,score,no_of_batsmen,venue_enc,batting_team_enc,bowling_team_enc
0,1,61,3,18,4,4
1,1,61,5,22,4,8
2,1,40,5,19,4,6
3,2,51,4,14,4,1
4,2,31,6,26,4,5


### Test Train Split

In [33]:
used_data = ordinal_enc_model_data.copy()# data used to train the model finally
X = used_data.drop(columns=['score'])
y = used_data['score']

In [34]:
X.head()

,innings,no_of_batsmen,venue_enc,batting_team_enc,bowling_team_enc
0,1,3,18,4,4
1,1,5,22,4,8
2,1,5,19,4,6
3,2,4,14,4,1
4,2,6,26,4,5


In [35]:
y.head()

0    61
1    61
2    40
3    51
4    31
Name: score, dtype: int32

In [36]:
# CODE for Date based split 
# X_train = enc_model_data[enc_model_data['start_date'] < 2018]
# X_test = enc_model_data[enc_model_data['start_date'] >= 2018]

# y_train = np.array(X_train["score"])
# y_test = np.array(X_test["score"])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,shuffle=True, random_state=42)

In [37]:
#Shape of the Dataset
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (1013, 5)
y_train shape:  (1013,)
X_test shape:  (254, 5)
y_test shape:  (254,)


In [41]:
# Defining the a metric Calculation Function to Calcultae the Model Performance
def metrics_scores(y_test,pred):
    print("MAE: ", metrics.mean_absolute_error(y_test, pred))
    print("MSE: ", metrics.mean_squared_error(y_test, pred))
    print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, pred)))
    print("R2 Score: ",reg_model.score(X_test,y_test))
    print("RSS( Residual Sum of Squares): ",metrics.mean_squared_error(y_test, reg_model.predict(X_test))*X_test.shape[0])

### Preparing the Model

#### 1) Linear Regresison Model

In [44]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Model Perfomance
metrics_scores(y_test,reg_model.predict(X_test))

MAE:  8.076901949914161
MSE:  110.25136641989164
RMSE:  10.50006506741228
R2 Score:  0.2014244446698389
RSS( Residual Sum of Squares):  28003.847070652475


> **Predicting Randomly**

In [46]:
for i in range(15):
    d = np.array([X_test.iloc[i,:]])
    true_value = y_test.iloc[i]
    pred_value = reg_model.predict(d)
    print(f"{i}. True Value: {true_value}         Prdicted Value: {pred_value}")

0. True Value: 29         Prdicted Value: [46.85783886]
1. True Value: 38         Prdicted Value: [43.7427609]
2. True Value: 47         Prdicted Value: [53.71580386]
3. True Value: 22         Prdicted Value: [41.49419332]
4. True Value: 41         Prdicted Value: [51.64434626]
5. True Value: 45         Prdicted Value: [46.14402441]
6. True Value: 45         Prdicted Value: [40.28273179]
7. True Value: 39         Prdicted Value: [49.31392568]
8. True Value: 25         Prdicted Value: [36.02365565]
9. True Value: 66         Prdicted Value: [54.88327603]
10. True Value: 43         Prdicted Value: [46.75902618]
11. True Value: 53         Prdicted Value: [47.88475269]
12. True Value: 17         Prdicted Value: [41.69181868]
13. True Value: 40         Prdicted Value: [50.55834685]
14. True Value: 15         Prdicted Value: [42.85345511]


#### 2) Ridge Regression

In [47]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge = Ridge()
alpha_values = {'alpha':[1e-15, 1e-10, 1e-8, 1e-3, 1e-2, 1, 5, 10, 20, 30 , 35, 40]}
ridge_regressor = GridSearchCV(ridge, alpha_values, scoring = "neg_mean_squared_error", cv = 5)
ridge_regressor.fit(X_train, y_train)

# Model Perfomance
metrics_scores(y_test,ridge_regressor.predict(X_test))

MAE:  8.075499181611896
MSE:  110.25393587587322
RMSE:  10.500187420987933
R2 Score:  0.2014244446698389
RSS( Residual Sum of Squares):  28003.847070652475


In [48]:
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 10}
-124.20980965386825


> **Predicting Randomly**

In [49]:
#Random Prediction
for i in range(15):
    d = np.array([X_test.iloc[i,:]])
    true_value = y_test.iloc[i]
    pred_value = ridge_regressor.predict(d)
    print(f"{i}. True Value: {true_value}         Prdeicted Value: {pred_value}")

0. True Value: 29         Prdeicted Value: [46.88326499]
1. True Value: 38         Prdeicted Value: [43.80467249]
2. True Value: 47         Prdeicted Value: [53.63478569]
3. True Value: 22         Prdeicted Value: [41.51721305]
4. True Value: 41         Prdeicted Value: [51.59072204]
5. True Value: 45         Prdeicted Value: [46.21020979]
6. True Value: 45         Prdeicted Value: [40.3039344]
7. True Value: 39         Prdeicted Value: [49.33968642]
8. True Value: 25         Prdeicted Value: [36.15369079]
9. True Value: 66         Prdeicted Value: [54.80107857]
10. True Value: 43         Prdeicted Value: [46.78539509]
11. True Value: 53         Prdeicted Value: [47.83567707]
12. True Value: 17         Prdeicted Value: [41.71295284]
13. True Value: 40         Prdeicted Value: [50.54482936]
14. True Value: 15         Prdeicted Value: [42.91062016]


#### 3) Random Forest Regressor

In [50]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, y_train)

#Model Performance 
metrics_scores(y_test,regr.predict(X_test))

MAE:  8.305126288955865
MSE:  117.4638702725596
RMSE:  10.838075026154765
R2 Score:  0.2014244446698389
RSS( Residual Sum of Squares):  28003.847070652475


> **Predicting Randomly**

In [51]:
#Random Prediction for Anrun Jatley Stadium
for i in range(len(X_test[X_test["venue_enc"] == 27])):
    d = np.array([X_test[X_test["venue_enc"] == 27].iloc[i,:]])
    true_value = y_test.iloc[i]
    pred_value = regr.predict(d)
    print(f"Array is {d} and {i}. True Value: {true_value}         Prdeicted Value: {pred_value}")

Array is [[ 2  3 27  7  2]] and 0. True Value: 29         Prdeicted Value: [48.605813]
Array is [[ 1  3 27  5  3]] and 1. True Value: 38         Prdeicted Value: [48.605813]
Array is [[ 2  2 27  3  3]] and 2. True Value: 47         Prdeicted Value: [52.68852512]
Array is [[ 2  2 27  5  3]] and 3. True Value: 22         Prdeicted Value: [52.68852512]
Array is [[ 2  2 27  5  3]] and 4. True Value: 41         Prdeicted Value: [52.68852512]
Array is [[ 2  2 27  3  3]] and 5. True Value: 45         Prdeicted Value: [52.68852512]
Array is [[ 2  3 27  2  3]] and 6. True Value: 45         Prdeicted Value: [48.605813]
Array is [[ 1  3 27  7  7]] and 7. True Value: 39         Prdeicted Value: [48.56363835]
Array is [[ 2  4 27  3  3]] and 8. True Value: 25         Prdeicted Value: [42.96132753]
Array is [[ 2  3 27  7  7]] and 9. True Value: 66         Prdeicted Value: [48.56363835]
Array is [[ 1  4 27  7  8]] and 10. True Value: 43         Prdeicted Value: [42.83674055]
Array is [[ 1  4 27  7  4]

### Doing Some Manual Predictions on Real Data of IPL 2021

In [53]:
#Order is :innings	no_of_batsmen	venue_enc	batting_team_enc	bowling_team_enc
inputArray = np.array([[2,2,27,7,2]])
print("Regression: ", reg_model.predict(inputArray))
print("Ridge: ", ridge_regressor.predict(inputArray))
print("Random Forest: ", regr.predict(inputArray))

Regression:  [55.48021026]
Ridge:  [55.39289458]
Random Forest:  [52.68852512]


> #### By Observing the Performance of Modesl on test dataset and the real time data predictions, the Ridge Regressor Model is selected

###  Dumping the Model into a Pickle File

In [54]:
import joblib

In [55]:
filename = 'ordinal_enc_ridge_regressor_batsmen.pkl'
joblib.dump(ridge_regressor, filename)

['ordinal_enc_ridge_regressor_batsmen.pkl']

#### Predicting using Model

In [57]:
#prediction form the joblib model
filename_import = 'ordinal_enc_ridge_regressor_batsmen.pkl'
my_model = joblib.load(filename_import)
inputArray = np.array([[2,2,27,7,2]])
my_model.predict(inputArray)

array([55.39289458])

**--------------------------------------------------------------------------------------------- END ----------------------------------------------------------------------------------------------------**